### Generating the MnAu2 cell structure inputs for the QE calculations

Read in an initial .cif file (6-atoms) and expand to 24 atoms cell by translating in the x,y,z directions to increase the number of Mn atoms. This allows for more magnetic patterns in the Mn layers. Saved to file as a .cif.

The pattern matrices and coordinates saved to file are used to generate the input for calculating configuation energies.

Total configuration energies and the pattern matrix are inputs for the exchange model calculation of the exchange parameters. 

In [389]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
import itertools
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression


from ruamel.yaml import YAML
yaml = YAML()

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

kB = 1000 * Boltzmann / electron_volt

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

### Starting with 6 atom cell:

In [233]:
cif_filepath = "data/mnau2-6.cif"
structure_mnau26 = from_file(structure_file=cif_filepath)
print(structure_mnau26)

Full Formula (Mn2 Au4)
Reduced Formula: MnAu2
abc   :   3.394839   3.394839   8.880636
angles:  90.000000  90.000000  90.000000
Sites (6)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Au    0.5  0.5  0.840319
  2  Au    0.5  0.5  0.159681
  3  Mn    0.5  0.5  0.5
  4  Au    0    0    0.340319
  5  Au    0    0    0.659681


### Getting the neighbor information to calculate the exchange parameters:

Magnetic structure read in from the 24 atom Mn8Au16 structure with determined unique (collinear) patterns.

In [390]:
cif_filepath1 = "data/mn8au16_collinear.cif"
cif_filepath2 = "data/mn8au16_1x1x4.cif"
structure_mnau2_c = from_file(structure_file=cif_filepath1)
structure_mnau2_z = from_file(structure_file=cif_filepath2)
#print(structure_mnau2_c)
#print(structure_mnau2_z)

/home/ssilayi/anaconda3/lib/python3.6/site-packages/pymatgen/core/lattice.py:694: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  result = np.linalg.lstsq(q.T, p.T)[0].T


Read in and rearrange the pattern matrices to match the atomic sequence in the structure and create a pattern matrix useable in the exchange model calculation.  

In [325]:
def pattern_tables( layer, p_type):
    n_row, n_column = layer.shape
    au_layer = pd.DataFrame(np.zeros((2,n_column)))
    patterns = pd.concat([ layer[0:1],au_layer,
                           layer[1:2],au_layer,
                           layer[2:3],au_layer,
                           layer[3:4],au_layer,
                           layer[4:5],au_layer,
                           layer[5:6],au_layer,
                           layer[6:7],au_layer,
                           layer[7:8],au_layer], ignore_index=True)
    
    p_types =[]
    p_types.append("nm")
    index = np.arange(1,len(patterns.columns))
    for i in index:
        p_types.append(p_type+str(i))
    patterns.columns = p_types
    
    return patterns

c_layer=pd.read_csv('data/mg_collinear_patterns', header=None)
c_patterns = pattern_tables( c_layer, "c-")

z_layer=pd.read_csv('data/mg_1x1x4_patterns', header=None)
z_patterns = pattern_tables( z_layer, "z-")

### Calculate the neighbor energies from the magnetic patterns:

In [393]:
def pattern_model(pattern_structure, pattern_mg):
    pattern_neighbor_data = count_neighbors(cell_structure=pattern_structure, r=8.8)
    exchange_model = build_model(magnetic_patterns=pattern_mg, neighbor_data=pattern_neighbor_data)
    #drop the Au atoms
    exchange_model = exchange_model[exchange_model.columns.drop(list(exchange_model.filter(regex='Au')))]
    
    return exchange_model
    
    
exchange_model_c = pattern_model(structure_mnau2_c, c_patterns)
exchange_model_z = pattern_model(structure_mnau2_z, z_patterns)

#combine different cells
exchange_model_all = pd.concat([exchange_model_c, exchange_model_z])
#exchange_model_all

#### Pre-process: Check that magnetic moment is not lost and that energy values are unique for the different patterns within a tolerance of 0.1 meV

 Calculated energies for different patterns using QE:

In [379]:
#Check that all energies are not within the tolerance value of each other
def drop_duplicates(qe_energies):
    
    qe_energies.columns = ["pattern", "num_sites", "total_energy" ]
    ryd_to_ev = 13.605693009 #eV 
    tol = 0.0001 #eV

    #convert to eV
    qe_energies["total_energy"] = qe_energies["total_energy"]*ryd_to_ev

    for i in range(len(qe_energies)-1):
        for j in range(1+1,len(qe_energies)):
            if (i != j):
                diff = abs(qe_energies["total_energy"][i] - qe_energies["total_energy"][j])
                if (  diff  <= tol ):
                    qe_energies["pattern"][j] = qe_energies["pattern"][i]
                    #print (i,j, qe_energies["total_energy"][i], qe_energies["total_energy"][j], diff)
                
                
    #drop repeating energies            
    qe_energies = qe_energies.reset_index(drop=True).drop_duplicates(["pattern"], keep="first")           
 
    #convert back to Rydberg
    qe_energies["total_energy"] = qe_energies["total_energy"]/ryd_to_ev
    
    return qe_energies

#Calculated energies from QE for the different patterns are tabulated and read in
qe_energies1 = pd.read_csv('data/energy.out.collinear.csv', header=None)
qe_energies_c = drop_duplicates(qe_energies1) 

qe_energies2 = pd.read_csv('data/energy.out.1x1x4.csv', header=None)
qe_energies_z = drop_duplicates(qe_energies2)    

#combine all energies
qe_energies_all = pd.concat([qe_energies_c, qe_energies_z])
qe_energies_all = qe_energies_all.reset_index(drop=True).drop_duplicates(["pattern"], keep="first") 
#qe_energies_all


/home/ssilayi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Checking magnetic moments are conserved:

In [396]:
def mg_moments_check(mg_moments, qe_energies):
    n_row, n_column = mg_moments.shape
    mg_moments = pd.DataFrame(round (abs(mg_moments/4.0)).sum(axis=0), columns=["total_moment"])
    mg_moments = pd.concat ([qe_energies['pattern'], mg_moments['total_moment']], axis=1)
    return mg_moments[(mg_moments['total_moment'] != n_row ) & (mg_moments['pattern'] != "nm") ] #.head()
    
#read in magnetic moments and return any that don't have the expected total magnetic moment (18)
mg_moments_c = pd.read_csv('data/mg_moment.collinear',sep='\t', header=None)
zero_moments_c = mg_moments_check(mg_moments_c, qe_energies_c)
#zero_moments_c

mg_moments_z = pd.read_csv('data/mg_moment.1x1x4',sep='\t', header=None)
zero_moments_z = mg_moments_check(mg_moments_z, qe_energies_z)
#zero_moments_z

### Building the Model Matrix:
Calculated energies and values from neighbor model matched according to pattern type. 
The energies are normalized relative to the "nm" type (spin 0).

In [397]:
def ModelMatrix(qe_energies, exchange_model):
    model_matrix = qe_energies \
        .assign(energy = lambda x:
            (x["total_energy"] -
             np.float64(x.query("pattern == 'nm'").loc[:, "total_energy"])) *
            1000 * units.Ha_to_eV / x["num_sites"])  \
        .merge(exchange_model, on=["pattern"]) \
        .query("pattern != 'nm'") \
        .assign(J1 = lambda x: x["J1_MnMn"]) \
        .assign(J2 = lambda x: x["J2_MnMn"]) \
        .assign(J3 = lambda x: x["J3_MnMn"]) \
        .assign(J4 = lambda x: x["J4_MnMn"]) \
        .assign(J5 = lambda x: x["J5_MnMn"]) \
        .assign(J6 = lambda x: x["J6_MnMn"]) \
        .assign(J7 = lambda x: x["J7_MnMn"]) \
        .assign(J8 = lambda x: x["J7_MnMn"]) \
        .loc[:, ["pattern", "energy", "J1", "J2", "J3", "J4", "J5" , "J6", "J7", "J8"]]

    return model_matrix

model_matrix = ModelMatrix(qe_energies_all, exchange_model_all)
#model_matrix

### Fitting to get final parameters:

Final matrix of energies and exchange model is used in a fit function to get the values of the J parameters.
2 results are compared, and values are in units of Rydberg.

In [398]:
smf_exchange_fit = smf.ols(data=model_matrix, formula="energy ~ J1 + J2 + J3 + J4 +J5 + J6 + J7 + J8").fit()
smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
#smf_exchange_parameters

In [399]:
lm = LinearRegression()
lm_exchange_fit = lm.fit(X=model_matrix[["J1", "J2", "J3", "J4", "J5", "J6", "J7", "J8"]], y=model_matrix["energy"])
lm_exchange_parameters = pd.DataFrame({
    "sklearn": [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],
                lm_exchange_fit.coef_[3], lm_exchange_fit.coef_[4],lm_exchange_fit.coef_[5],
                lm_exchange_fit.coef_[6], lm_exchange_fit.coef_[7]]},
    index=["Intercept", "J1", "J2", "J3", "J4", "J5", "J6", "J7", "J8"],
)
#lm_exchange_parameters

In [400]:
exchange_parameters = pd.concat([smf_exchange_parameters,lm_exchange_parameters], axis=1)
#save to file    
exchange_parameters.to_csv("data/mnau2_exchange_parameters.collinear.txt", sep=' ')
exchange_parameters

,stats_models,sklearn
Intercept,-2340.698546,-2340.698546
J1,-4.622895,-4.622895
J2,-12.230765,-12.230765
J3,-1.916237,-1.916237
J4,-16.408480,-16.408480
J5,2.393551,2.393551
J6,-9.245790,-9.245790
J7,-1.916237,-1.916237
J8,-1.916237,-1.916237
